In [2]:
from pyspark.sql import SparkSession #Import para sessão spark
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, FloatType #Imports para estrutura e tipos

from delta import *

import logging

logging.getLogger("py4j").setLevel(logging.DEBUG)

In [3]:
#Confiraçã0 da Sessão spark
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

25/04/23 14:05:50 WARN Utils: Your hostname, Guichard resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/23 14:05:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/guichard/.cache/pypoetry/virtualenvs/engenharia-de-dados-apache-spark-FSwoO1eP-py3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/guichard/.ivy2/cache
The jars for the packages stored in: /home/guichard/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0de7995c-a28c-4f26-8f8c-ccf6701a9bf2;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 407ms :: artifacts dl 14ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

In [3]:
spark # Sessão

In [4]:
#Simulação base de clientes
data = [
    ("ID001", "CLIENTE_X","SP","ATIVO",   250000.00),
    ("ID002", "CLIENTE_Y","SC","INATIVO", 400000.00),
    ("ID003", "CLIENTE_Z","DF","ATIVO",   1000000.00)
]

schema = (
    StructType([
        StructField("ID_CLIENTE",     StringType(),True), #True = Pode ser nulo
        StructField("NOME_CLIENTE",   StringType(),True),
        StructField("UF",             StringType(),True),
        StructField("STATUS",         StringType(),True),
        StructField("LIMITE_CREDITO", FloatType(), True)
    ])
)

df = spark.createDataFrame(data=data,schema=schema) #Dataframe

df.show(truncate=False) #Permite visualizar os dados sem quebrar

+----------+------------+---+-------+--------------+
|ID_CLIENTE|NOME_CLIENTE|UF |STATUS |LIMITE_CREDITO|
+----------+------------+---+-------+--------------+
|ID001     |CLIENTE_X   |SP |ATIVO  |250000.0      |
|ID002     |CLIENTE_Y   |SC |INATIVO|400000.0      |
|ID003     |CLIENTE_Z   |DF |ATIVO  |1000000.0     |
+----------+------------+---+-------+--------------+



# Grava Delta Table

In [5]:
# Gera uma delta table a partir do dataframe

(
    df
    .write
    .format("delta")
    .mode("overwrite")
    .save("./RAW/CLIENTES")
)

25/04/23 14:06:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

# Simulação de cenário (NOVOS DADOS)

In [ ]:
new_data = [
    ("ID001","CLIENTE_X","SP","INATIVO", 0.00),
    ("ID002","CLIENTE_Y","SC","ATIVO",   400000.00),
    ("ID004","CLIENTE_Z","DF","ATIVO",   5000000.00)
] # Novo lote da partição do data lake

df_new = spark.createDataFrame(data=new_data, schema=schema) # Cria um dataframe com os novo dados

df_new.show() # Mostra o dataframe

# UPSERT / MERGE 

### OBS: Não ler delta tables com read parquet

In [ ]:
#Criação da delta table
delta_table= DeltaTable.forPath(spark, "./RAW/CLIENTES")

In [ ]:
delta_table.toDF().show() #Transforma a leitura em um dataframe e da output

In [ ]:
# MERGE
(
    delta_table.alias("dados_atuais")
    .merge(
        df_new.alias("novos_dados"),
        "dados_atuais.ID_CLIENTE = novos_dados.ID_CLIENTE" # Relacionamento de chaves
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll() # All se refere a todos os dados, caso queira especificar, é possível remover o all colocar apenas o desejado.
    .execute() # executa
)

In [ ]:
delta_table.toDF().show()

In [ ]:
# Delete
delta_table.delete("LIMITE_CREDITO < 400000.0") #Condição (Não exclui fisicamente)

In [ ]:
delta_table.toDF().show()

# Operações com delta table (CRUD)

# CRIAÇÃO DE TABELA

In [5]:
# Schema
schema_clientes = (
    StructType([
        StructField("id",     StringType(),False), #True = Pode ser nulo
        StructField("nome_cliente",   StringType(),False),
        StructField("uf",             StringType(),False),
        StructField("status",         StringType(),False),
        StructField("cidade",         StringType(),False),
        StructField("limite_credito", FloatType(), True)
    ])
)

In [3]:
# Criação da tabela cliente_delta
spark.sql(
     """
     CREATE TABLE IF NOT EXISTS default.clientes_delta (
         id INT NOT NULL,
         nome_cliente STRING NOT NULL,
         uf CHAR(2) NOT NULL,
         status BOOLEAN NOT NULL,
         limite_credito FLOAT NOT NULL
     ) USING delta
     """
)

25/04/23 13:28:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

DataFrame[]

In [4]:
# Relaiza um select na tabela criada e faz o output
spark.sql(
    """
    SELECT * from default.clientes_delta
    """
).show()

+---+------------+---+------+--------------+
| id|nome_cliente| uf|status|limite_credito|
+---+------------+---+------+--------------+
+---+------------+---+------+--------------+



In [8]:
spark.sql("SHOW TABLES IN default").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|  default|clientes_delta|      false|
+---------+--------------+-----------+



In [9]:
# Criação da delta table para cliente delta
cliente = DeltaTable.forPath(spark, "./spark-warehouse/clientes_delta") 

In [10]:
cliente.toDF().show() # Transforma a leitura em um Dataframe

+---+------------+---+------+--------------+
| id|nome_cliente| uf|status|limite_credito|
+---+------------+---+------+--------------+
+---+------------+---+------+--------------+



In [ ]:
# mostra o histórico
cliente.history().show()

# INSERT

In [11]:
#INSERT
spark.sql(
    """
        INSERT INTO default.clientes_delta VALUES 
        (1, 'Jean Guichard²', 'RS', false, 10000000.0), 
        (2, 'Lucas da Rosa', 'SC', true, 1000000.0), 
        (3, 'Matheus Daminelli', 'SC', true, 4000000.0)   
    """
)

DataFrame[]

In [ ]:
spark.sql(
    """
        INSERT INTO default.clientes_delta VALUES    
        (4, 'Teste da Silva', 'SP', true, 8000000.0, 'Testenópolis')   
    """
)

In [23]:
spark.sql(
    """
    SELECT * from default.clientes_delta
    """
).show()

+---+-----------------+---+------+--------------+--------+
| id|     nome_cliente| uf|status|limite_credito|  cidade|
+---+-----------------+---+------+--------------+--------+
|  3|Matheus Daminelli| SC|  true|     4000000.0|Criciúma|
|  2|    Lucas da Rosa| SC|  true|     1000000.0| Tubarão|
|  1|   Jean Guichard²| RS| false|         1.0E7|  Torres|
+---+-----------------+---+------+--------------+--------+



In [14]:
# Mostra o histórico sem quebrar dados
cliente.history().show(truncate=False)

+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                          |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-04-22 20:38:03.791|NULL  |NULL    |WRITE 

# ALTERANDO TABELA

In [15]:
# Adicionando coluna
spark.sql(
    """
    alter table default.clientes_delta add column cidade STRING
    """
)

DataFrame[]

In [17]:
spark.sql(
    """
    SELECT * from default.clientes_delta
    """
).show()

+---+-----------------+---+------+--------------+------+
| id|     nome_cliente| uf|status|limite_credito|cidade|
+---+-----------------+---+------+--------------+------+
|  3|Matheus Daminelli| SC|  true|     4000000.0|  NULL|
|  1|   Jean Guichard²| RS| false|         1.0E7|  NULL|
|  2|    Lucas da Rosa| SC|  true|     1000000.0|  NULL|
+---+-----------------+---+------+--------------+------+



# UPDATE

In [19]:
# Update
spark.sql("""
    UPDATE default.clientes_delta
    SET cidade = CASE
        WHEN id = 1 THEN 'Torres'
        WHEN id = 2 THEN 'Tubarão'
        WHEN id = 3 THEN 'Criciúma'
        ELSE cidade
    END
""")

DataFrame[num_affected_rows: bigint]

In [20]:
spark.sql(
    """
    SELECT * from default.clientes_delta
    """
).show()

+---+-----------------+---+------+--------------+--------+
| id|     nome_cliente| uf|status|limite_credito|  cidade|
+---+-----------------+---+------+--------------+--------+
|  3|Matheus Daminelli| SC|  true|     4000000.0|Criciúma|
|  2|    Lucas da Rosa| SC|  true|     1000000.0| Tubarão|
|  1|   Jean Guichard²| RS| false|         1.0E7|  Torres|
+---+-----------------+---+------+--------------+--------+



In [ ]:
# Novo dado
new_data_cliente = [
    ("4","Teste da Silva","SP","false", 100.00, 'São Paulo'),
] # Novo lote da partição do data lake

df_new_cliente = spark.createDataFrame(data=new_data_cliente, schema=schema_cliente) # Cria um dataframe com os novo dados

df_new_cliente.show()

In [ ]:
(
    cliente.alias("dados_atuais")
    .merge(
        df_new_cliente.alias("novos_dados"),
        "dados_atuais.id = novos_dados.id" # Relacionamento de chaves
    )
    .whenMatchedUpdateAll()
    .whenNotMatchedInsertAll() # All se refere a todos os dados, caso queira especificar, é possível remover o all colocar apenas o desejado.
    .execute() # executa
)
cliente.toDF().show()

### Verifica se é uma delta table

In [22]:
DeltaTable.isDeltaTable(spark, "spark-warehouse/clientes_delta")

True

# DELETE

In [ ]:
# Delete
spark.sql(
    """
        DELETE FROM default.clientes_delta
        WHERE uf = 'RS' AND status = false
    """
)

In [ ]:
spark.sql(
    """
    SELECT * from default.clientes_delta
    """
).show()

In [ ]:
cliente.delete("limite_credito < 200.0")

In [ ]:
cliente.toDF().show()

# Exibição em Display

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [ ]:
spark.sql('describe HISTORY cliente_delta').show(truncate=False);